In [50]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [51]:
train_df=pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test_df=pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")

In [52]:
train_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [53]:
train_df.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [54]:
train_df.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [55]:
ID =test_df['PassengerId']
train_df=train_df.drop(['PassengerId','Name'],axis=1)
test_df=test_df.drop(['PassengerId','Name'],axis=1)

In [56]:
categorical_cols = train_df.select_dtypes(include=['object']).columns
numerical_cols =  train_df.select_dtypes(include=[np.number]).columns

In [57]:
def mode_imputation(df, categorical_cols):
    for col in categorical_cols:
        mode_val = df[col].mode()[0]
        df[col].fillna(mode_val, inplace=True)

def median_imputation(df, numerical_cols):
    for col in numerical_cols:
        median_val = df[col].median()
        df[col].fillna(median_val, inplace=True)

In [58]:
#for train data set
mode_imputation(train_df,categorical_cols)
median_imputation(train_df,numerical_cols)

#for test datd set
mode_imputation(test_df,categorical_cols)
median_imputation(test_df,numerical_cols)


/tmp/ipykernel_33/3568324371.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mode_val, inplace=True)
/tmp/ipykernel_33/3568324371.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col].fillna(mode_val, inplace=True)
/tmp/ipykernel_33/3568324371.py:9: FutureWarning: A value is trying to b

In [59]:
train_df.isnull().sum()

HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64

**label encoding**

In [60]:
from sklearn.preprocessing import LabelEncoder

labelencode=LabelEncoder()

for column in train_df.columns:
    if train_df[column].dtype=="object" or train_df[column].dtype=="bool":
        train_df[column]= labelencode.fit_transform(train_df[column])
        
for column in test_df.columns:
    if test_df[column].dtype=="object" or test_df[column].dtype=="bool":
        test_df[column]= labelencode.fit_transform(test_df[column])

In [61]:
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [62]:
X=train_df.drop('Transported',axis=1)
y=train_df['Transported']

In [63]:
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2)

In [64]:
clsfrs=[GradientBoostingClassifier(n_estimators=300,max_depth=4,learning_rate=0.1,subsample=1.0),CatBoostClassifier(n_estimators=300,max_depth=4,verbose=0,learning_rate=0.1,subsample=1.0),XGBClassifier(n_estimators=300,max_depth=4,learning_rate=0.1,subsample=1.0)]

In [65]:
for classfier in clsfrs:
    gbst =classfier
    gbst.fit(Xtrain,ytrain)
    gbpred=gbst.predict(Xtest)
    accracy=accuracy_score(ytest,gbpred)
    print(accracy)



0.7929844738355377
0.7981598619896493
0.8062104657849338


In [66]:
pred=CatBoostClassifier(n_estimators=300,max_depth=4,verbose=0,learning_rate=0.1,subsample=1.0)
pred.fit(Xtrain,ytrain)
predict=pred.predict(test_df)

final=predict.astype(bool)

In [67]:
submission_df = pd.DataFrame({
    'PassengerId': ID,
    'Transported': final
})

In [68]:
submission_df.to_csv('submission.csv', index = None)